<div style="color: navy; text-align: center; font-family: 'Georgia', serif; font-size: 32px; font-weight: bold;">
    Description
</div>

<div style="color: green; text-align: left; font-family: 'Calibri', sans-serif; font-size: 32px;">
    Hello, dear colleagues!
</div>

<div style="color: dark-blue; text-align: left; font-family: 'Calibri', sans-serif; font-size: 24px;">
    This notebook was made in educational purposes, every model was created by myself using official documentation.
    I hope it will be useful for other beginners in ML.
    If you have check it, please, upvote!
    If you have any questions or suggestions about improvements, leave it in comment section.
</div>


<div style="color: Red; text-align: center; font-family: 'Georgia', serif; font-size: 32px; font-weight: bold;">
    About this notebook
</div>

<div style="color: dark-blue; text-align: left; font-family: 'Calibri', sans-serif; font-size: 24px;">
    The aim of this short research is to investigate how different models and their combinations can perform in Spaceship Titanic task.
</div>


<div style="color: dark-blue; text-align: left; font-family: 'Calibri', sans-serif; font-size: 24px;">
    In this work I consider 5 models: <br>
        &nbsp&nbsp&nbsp&nbsp 1. XGBOOST Classifier<br>
        &nbsp&nbsp&nbsp&nbsp 2. Random Forest Classifier<br>
        &nbsp&nbsp&nbsp&nbsp 3. KNN Classifier <br>
        &nbsp&nbsp&nbsp&nbsp 4. Support Vector Machines Classifier<br>
        &nbsp&nbsp&nbsp&nbsp 5. Linear Discriminant Analysis <br>
    
    In future, I plan to add some other models here.
</div>


<div style="color: navy; text-align: center; font-family: 'Georgia', serif; font-size: 42px; font-weight: bold;">
    The best score is 0.80196
</div>

<div style="color: Red; text-align: center; font-family: 'Georgia', serif; font-size: 32px; font-weight: bold;">
    Let's start
</div>

# Libraries import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import plotly
import plotly.express as px
from sklearn.model_selection import train_test_split

# Data import

In [2]:
train_data = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test_data = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [3]:
sample_submission = pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')
sample_submission

,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,False
4,0023_01,False
...,...,...
4272,9266_02,False
4273,9269_01,False
4274,9271_01,False
4275,9273_01,False


In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


# Data Preparing

In [5]:
train_data.drop('PassengerId',axis=1,inplace=True)

In [6]:
arr = []
for i in range(len(train_data)):
    try:   
        arr.append(train_data['Name'][i].split(' ')[1])
    except:
        arr.append(f'NoName_{i}')

In [7]:
# train_data['SecondName'] = arr
train_data.drop('Name',axis=1,inplace=True)

In [8]:
arr = []
for i in range(len(test_data)):
    try:   
        arr.append(test_data['Name'][i].split(' ')[1])
    except:
        arr.append(f'NoName_{i}')

In [9]:
# test_data['SecondName'] = arr
test_data.drop('Name',axis=1,inplace=True)

In [10]:
train_data

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False
8689,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,False
8690,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,True
8691,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False


In [11]:
train_acc = []

In [12]:
print(len(train_data[train_data.Transported == False]))
print(len(train_data[train_data.Transported == True]))

4315
4378


In [13]:
val = 0.2

In [14]:
i = 0
while True:
    X, y = train_data.drop('Transported', axis=1), train_data['Transported']
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=val, random_state=i)
    unique_elements, counts = np.unique(np.array(y_train), return_counts=True)

    element_counts = dict(zip(unique_elements, counts))
        
    if abs(element_counts[False] - element_counts[True]) < 5:
        break
        
    i+=1
    if i==10000:
        break

In [15]:
X_train_copy, y_train_copy = X_train.copy(), y_train.copy()

In [16]:
print("Unique elements and count:")
for element, count in element_counts.items():
    print(f"{element}: {count}")
seed = i
print('seed', seed)

Unique elements and count:
False: 3476
True: 3478
seed 20


In [17]:
thr = 0.8
final_valid_test = y_valid[int(len(y_valid)*thr):]
final_valid_features = X_valid[int(len(X_valid)*thr):]
y_valid = y_valid[:int(len(y_valid)*thr)]
X_valid = X_valid[:int(len(X_valid)*thr)]

In [18]:
print(len(X_train))
print(len(X_valid))
print(len(final_valid_test))
print(len(X_valid)+len(final_valid_test))

6954
1391
348
1739


In [19]:
object_columns = list(X.select_dtypes(include=['object']).columns)
object_columns

['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP']

# Model 1. XGBOOST

In [20]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score, precision_score

best_params = {
    'n_estimators': 5000,
    'max_depth': 10,
    'learning_rate': 0.01,
    'subsample': 0.2,
}

model_1 = xgb.XGBClassifier(**best_params, eval_metric='auc', tree_method="hist", objective='binary:logistic', enable_categorical=True)

for feature in object_columns:  
    X_train[feature] = X_train[feature].astype("category")
    X_valid[feature] = X_valid[feature].astype("category")

model_1.fit(X_train, y_train)

y_train_pred = model_1.predict(X_train)

y_valid_pred = model_1.predict(X_valid)

print("Metrics for the training set:")
print(f"Accuracy: {accuracy_score(y_train, y_train_pred)}")
print(f"F1 Score: {f1_score(y_train, y_train_pred)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_train, y_train_pred)}")
print(f"Recall: {recall_score(y_train, y_train_pred)}")
print(f"Precision: {precision_score(y_train, y_train_pred)}")

print("\nMetrics for the validation set:")
print(f"Accuracy: {accuracy_score(y_valid, y_valid_pred)}")
print(f"F1 Score: {f1_score(y_valid, y_valid_pred)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_valid, y_valid_pred)}")
print(f"Recall: {recall_score(y_valid, y_valid_pred)}")
print(f"Precision: {precision_score(y_valid, y_valid_pred)}")

train_acc.append(accuracy_score(y_train, y_train_pred))

Metrics for the training set:
Accuracy: 0.9272361230946218
F1 Score: 0.9295460874408243
Confusion Matrix:
[[3110  366]
 [ 140 3338]]
Recall: 0.9597469810235768
Precision: 0.9011879049676026

Metrics for the validation set:
Accuracy: 0.7778576563623293
F1 Score: 0.7861591695501732
Confusion Matrix:
[[514 140]
 [169 568]]
Recall: 0.7706919945725916
Precision: 0.8022598870056498


In [21]:
for feature in object_columns:  
    test_data[feature] = test_data[feature].astype("category")
    
test_predict = model_1.predict(test_data.drop('PassengerId',axis=1))

In [22]:
output_1 = pd.DataFrame({'PassengerId':test_data.PassengerId,
                       'Transported':test_predict})
# output_1['Transported'] = output_1['Transported'].replace({1: True, 0: False})

# Light GMB

In [23]:
import lightgbm as lgb
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score, precision_score
from sklearn.model_selection import train_test_split

best_params = {
    'boosting_type': 'dart',
    'objective': 'binary',
    'metric': 'accuracy',
    'n_estimators': 2000,
    'max_depth': 10,
    'learning_rate': 0.01,
    'subsample': 0.2,
    'num_leaves': 10  # Set the number of leaves

}

d_train = lgb.Dataset(X_train, label=y_train, categorical_feature=object_columns)

model_lgb = lgb.train(best_params, d_train)

y_train_pred = np.round(model_lgb.predict(X_train))
y_valid_pred = np.round(model_lgb.predict(X_valid))

print("Metrics for the training set:")
print(f"Accuracy: {accuracy_score(y_train, y_train_pred)}")
print(f"F1 Score: {f1_score(y_train, y_train_pred)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_train, y_train_pred)}")
print(f"Recall: {recall_score(y_train, y_train_pred)}")
print(f"Precision: {precision_score(y_train, y_train_pred)}")

print("\nMetrics for the validation set:")
print(f"Accuracy: {accuracy_score(y_valid, y_valid_pred)}")
print(f"F1 Score: {f1_score(y_valid, y_valid_pred)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_valid, y_valid_pred)}")
print(f"Recall: {recall_score(y_valid, y_valid_pred)}")
print(f"Precision: {precision_score(y_valid, y_valid_pred)}")


/opt/conda/lib/python3.10/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


[LightGBM] [Info] Number of positive: 3478, number of negative: 3476
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005011 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1662
[LightGBM] [Info] Number of data points in the train set: 6954, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500144 -> initscore=0.000575
[LightGBM] [Info] Start training from score 0.000575
Metrics for the training set:
Accuracy: 0.8140638481449526
F1 Score: 0.8194890409046489
Confusion Matrix:
[[2726  750]
 [ 543 2935]]
Recall: 0.8438757906843013
Precision: 0.7964721845318861

Metrics for the validation set:
Accuracy: 0.8044572250179727
F1 Score: 0.820580474934037
Confusion Matrix:
[[497 157]
 [115 622]]
Recall: 0.8439620081411127
Precision: 0.7984595635430038


In [24]:
for feature in object_columns:  
    test_data[feature] = test_data[feature].astype("category")
    
test_predict = np.round(model_lgb.predict(test_data.drop('PassengerId', axis=1)))

In [25]:
output_lgb = pd.DataFrame({'PassengerId':test_data.PassengerId,
                       'Transported':test_predict})

# Model 2. Random Forest

In [26]:
from sklearn.preprocessing import LabelEncoder
label_encoders = {}
for column in X_train.select_dtypes(include=['category']).columns:
    label_encoders[column] = LabelEncoder()
    X_train[column] = label_encoders[column].fit_transform(X_train[column])

In [27]:
from sklearn.preprocessing import LabelEncoder
label_encoders = {}
for column in X_valid.select_dtypes(include=['category']).columns:
    label_encoders[column] = LabelEncoder()
    X_valid[column] = label_encoders[column].fit_transform(X_valid[column])

In [28]:
from sklearn.preprocessing import LabelEncoder
label_encoders = {}
for column in test_data.select_dtypes(include=['category']).columns:
    label_encoders[column] = LabelEncoder()
    test_data[column] = label_encoders[column].fit_transform(test_data[column])

In [29]:
X_train = X_train.fillna(0)
X_valid = X_valid.fillna(0)
test_data = test_data.fillna(0)

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score, precision_score

best_params = {
    'n_estimators': 5000,
    'max_depth': 10,
    'min_samples_split': 2,
    'min_samples_leaf': 1,
    'bootstrap': True,
    'random_state': 42,  
}

model_2 = RandomForestClassifier(**best_params)

for feature in object_columns:  
    X_train[feature] = X_train[feature].astype("category")
    X_valid[feature] = X_valid[feature].astype("category")

model_2.fit(X_train, y_train)

y_train_pred = model_2.predict(X_train)

y_valid_pred = model_2.predict(X_valid)

print("Metrics for the training set:")
print(f"Accuracy: {accuracy_score(y_train, y_train_pred)}")
print(f"F1 Score: {f1_score(y_train, y_train_pred)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_train, y_train_pred)}")
print(f"Recall: {recall_score(y_train, y_train_pred)}")
print(f"Precision: {precision_score(y_train, y_train_pred)}")

print("\nMetrics for the validation set:")
print(f"Accuracy: {accuracy_score(y_valid, y_valid_pred)}")
print(f"F1 Score: {f1_score(y_valid, y_valid_pred)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_valid, y_valid_pred)}")
print(f"Recall: {recall_score(y_valid, y_valid_pred)}")
print(f"Precision: {precision_score(y_valid, y_valid_pred)}")

train_acc.append(accuracy_score(y_train, y_train_pred))

Metrics for the training set:
Accuracy: 0.858642507909117
F1 Score: 0.8619188088214637
Confusion Matrix:
[[2903  573]
 [ 410 3068]]
Recall: 0.8821161587119034
Precision: 0.842625652293326

Metrics for the validation set:
Accuracy: 0.7979870596693026
F1 Score: 0.8209050350541747
Confusion Matrix:
[[466 188]
 [ 93 644]]
Recall: 0.8738127544097694
Precision: 0.7740384615384616


In [31]:
test_predict = model_2.predict(test_data.drop('PassengerId',axis=1))

In [32]:
output_2 = pd.DataFrame({'PassengerId':test_data.PassengerId,
                       'Transported':test_predict})
output_2['Transported'] = output_2['Transported'].replace({True: 1, False: 0})

In [33]:
# output_2.to_csv('submission.csv',index=False)

In [34]:
output_2

,PassengerId,Transported
0,0013_01,1
1,0018_01,0
2,0019_01,1
3,0021_01,1
4,0023_01,0
...,...,...
4272,9266_02,1
4273,9269_01,0
4274,9271_01,1
4275,9273_01,1


# Model 3. KNN Classifier

In [35]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score, precision_score

n_neighbors = 3

model_3 = KNeighborsClassifier(n_neighbors=n_neighbors)

model_3.fit(X_train, y_train)

y_train_pred = model_3.predict(X_train)

y_valid_pred = model_3.predict(X_valid)

print("Metrics for the training set:")
print(f"Accuracy: {accuracy_score(y_train, y_train_pred)}")
print(f"F1 Score: {f1_score(y_train, y_train_pred)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_train, y_train_pred)}")
print(f"Recall: {recall_score(y_train, y_train_pred)}")
print(f"Precision: {precision_score(y_train, y_train_pred)}")

print("\nMetrics for the validation set:")
print(f"Accuracy: {accuracy_score(y_valid, y_valid_pred)}")
print(f"F1 Score: {f1_score(y_valid, y_valid_pred)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_valid, y_valid_pred)}")
print(f"Recall: {recall_score(y_valid, y_valid_pred)}")
print(f"Precision: {precision_score(y_valid, y_valid_pred)}")

train_acc.append(accuracy_score(y_train, y_train_pred))

Metrics for the training set:
Accuracy: 0.8607995398331896
F1 Score: 0.8605187319884726
Confusion Matrix:
[[3000  476]
 [ 492 2986]]
Recall: 0.8585393904542841
Precision: 0.8625072212593876

Metrics for the validation set:
Accuracy: 0.7361610352264558
F1 Score: 0.773875539125077
Confusion Matrix:
[[396 258]
 [109 628]]
Recall: 0.8521031207598372
Precision: 0.708803611738149


In [36]:
test_predict = model_3.predict(test_data.drop('PassengerId',axis=1))

In [37]:
output_3 = pd.DataFrame({'PassengerId':test_data.PassengerId,
                       'Transported':test_predict})
output_3['Transported'] = output_3['Transported'].replace({True: 1, False: 0})

# Model 4. 

In [38]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score, precision_score

C_value = 1.0
kernel_type = 'rbf'  
degree_value = 3  
gamma_value = 'scale'  
coef0_value = 0.0  
shrinking_enabled = True  
probability_enabled = True  
class_weights = None  
decision_function_shape = 'ovr'  

model_svm = SVC(C=C_value, kernel=kernel_type, degree=degree_value, gamma=gamma_value,
                coef0=coef0_value, shrinking=shrinking_enabled, probability=probability_enabled,
                class_weight=class_weights, decision_function_shape=decision_function_shape)


model_svm.fit(X_train, y_train)

y_train_pred = model_svm.predict(X_train)
y_valid_pred = model_svm.predict(X_valid)

print("Metrics for the training set:")
print(f"Accuracy: {accuracy_score(y_train, y_train_pred)}")
print(f"F1 Score: {f1_score(y_train, y_train_pred)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_train, y_train_pred)}")
print(f"Recall: {recall_score(y_train, y_train_pred)}")
print(f"Precision: {precision_score(y_train, y_train_pred)}")

print("\nMetrics for the validation set:")
print(f"Accuracy: {accuracy_score(y_valid, y_valid_pred)}")
print(f"F1 Score: {f1_score(y_valid, y_valid_pred)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_valid, y_valid_pred)}")
print(f"Recall: {recall_score(y_valid, y_valid_pred)}")
print(f"Precision: {precision_score(y_valid, y_valid_pred)}")

train_acc.append(accuracy_score(y_train, y_train_pred))

Metrics for the training set:
Accuracy: 0.7824273799252229
F1 Score: 0.8028664495114007
Confusion Matrix:
[[2360 1116]
 [ 397 3081]]
Recall: 0.8858539390454284
Precision: 0.73409578270193

Metrics for the validation set:
Accuracy: 0.7814521926671459
F1 Score: 0.8173076923076923
Confusion Matrix:
[[407 247]
 [ 57 680]]
Recall: 0.9226594301221167
Precision: 0.7335490830636462


In [39]:
test_predict = model_svm.predict(test_data.drop('PassengerId',axis=1))

In [40]:
output_4 = pd.DataFrame({'PassengerId':test_data.PassengerId,
                       'Transported':test_predict})
output_4['Transported'] = output_4['Transported'].replace({True: 1, False: 0})

# Model 5.

In [41]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score, precision_score


lda = LinearDiscriminantAnalysis(
    solver='svd',  
    shrinkage=None,  
    n_components=1,  
    priors=[0.5,0.5], 
    store_covariance=False,  
    
)

lda.fit(X_train, y_train)

y_train_pred = lda.predict(X_train)
y_valid_pred = lda.predict(X_valid)

accuracy_train = accuracy_score(y_train, y_train_pred)
f1_train = f1_score(y_train, y_train_pred)
conf_matrix_train = confusion_matrix(y_train, y_train_pred)
recall_train = recall_score(y_train, y_train_pred)
precision_train = precision_score(y_train, y_train_pred)

accuracy_valid = accuracy_score(y_valid, y_valid_pred)
f1_valid = f1_score(y_valid, y_valid_pred)
conf_matrix_valid = confusion_matrix(y_valid, y_valid_pred)
recall_valid = recall_score(y_valid, y_valid_pred)
precision_valid = precision_score(y_valid, y_valid_pred)

print("Metrics for the training set:")
print(f"Accuracy: {accuracy_score(y_train, y_train_pred)}")
print(f"F1 Score: {f1_score(y_train, y_train_pred)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_train, y_train_pred)}")
print(f"Recall: {recall_score(y_train, y_train_pred)}")
print(f"Precision: {precision_score(y_train, y_train_pred)}")

print("\nMetrics for the validation set:")
print(f"Accuracy: {accuracy_score(y_valid, y_valid_pred)}")
print(f"F1 Score: {f1_score(y_valid, y_valid_pred)}")
print(f"Confusion Matrix:\n{confusion_matrix(y_valid, y_valid_pred)}")
print(f"Recall: {recall_score(y_valid, y_valid_pred)}")
print(f"Precision: {precision_score(y_valid, y_valid_pred)}")

train_acc.append(accuracy_score(y_train, y_train_pred))

Metrics for the training set:
Accuracy: 0.7582686223756112
F1 Score: 0.7509998518737965
Confusion Matrix:
[[2738  738]
 [ 943 2535]]
Recall: 0.7288671650373778
Precision: 0.7745187901008249

Metrics for the validation set:
Accuracy: 0.7706685837526959
F1 Score: 0.7899934167215272
Confusion Matrix:
[[472 182]
 [137 600]]
Recall: 0.8141112618724559
Precision: 0.7672634271099744


In [42]:
test_predict = lda.predict(test_data.drop('PassengerId',axis=1))

In [43]:
output_5 = pd.DataFrame({'PassengerId':test_data.PassengerId,
                       'Transported':test_predict})
output_5['Transported'] = output_5['Transported'].replace({True: 1, False: 0})

# Searching for the best models combination

In [44]:
for feature in object_columns:  
    final_valid_features[feature] = final_valid_features[feature].astype("category")
    
# test_predict = model_1.predict(test_data.drop('PassengerId',axis=1))
p1 = model_1.predict(final_valid_features)
p6 = model_lgb.predict(final_valid_features)


for feature in object_columns:  
    X_train_copy[feature] = X_train_copy[feature].astype("category")
    
# test_predict = model_1.predict(test_data.drop('PassengerId',axis=1))
tr1 = model_1.predict(X_train_copy)
tr6 = model_lgb.predict(X_train_copy)


In [45]:
label_encoders = {}
for column in final_valid_features.select_dtypes(include=['category']).columns:
    label_encoders[column] = LabelEncoder()
    final_valid_features[column] = label_encoders[column].fit_transform(final_valid_features[column])
final_valid_features = final_valid_features.fillna(0)


label_encoders = {}
for column in X_train_copy.select_dtypes(include=['category']).columns:
    label_encoders[column] = LabelEncoder()
    X_train_copy[column] = label_encoders[column].fit_transform(X_train_copy[column])
X_train_copy = X_train_copy.fillna(0)

In [46]:
p2 = model_2.predict(final_valid_features)
p3 = model_3.predict(final_valid_features)
p4 = model_svm.predict(final_valid_features)
p5 = lda.predict(final_valid_features)

tr2 = model_2.predict(X_train_copy)
tr3 = model_3.predict(X_train_copy)
tr4 = model_svm.predict(X_train_copy)
tr5 = lda.predict(X_train_copy)


In [47]:
combinations = [
    (1, 2, 3), (1, 2, 4), (1, 2, 5), (1, 2, 6), (1, 3, 4), (1, 3, 5), (1, 3, 6), (1, 4, 5), (1, 4, 6), (1, 5, 6), (2, 3, 4), (2, 3, 5), (2, 3, 6), 
    (2, 4, 5), (2, 4, 6), (2, 5, 6), (3, 4, 5), (3, 4, 6), (3, 5, 6), (4, 5, 6),
    (1, 2, 3, 4, 5), (1, 2, 3, 4, 6), (1, 2, 3, 5, 6), (1, 2, 4, 5, 6), (1, 3, 4, 5, 6), (2, 3, 4, 5, 6),(1, 2, 3, 4)
]

In [48]:
model_predictions = [p1, p2, p3, p4, p5, p6]

check = pd.DataFrame({f'model{i+1}': model for i, model in enumerate(model_predictions)})

for col in check.columns:
    check[col] = check[col].replace({True: 1, False: 0})
    
check['model6'] = np.round(check['model6'])

for combo in combinations:
    col_name = f'composition_{"".join(map(str, combo))}'
    check[col_name] = check[list(map(lambda x: f'model{x}', combo))].mean(axis=1)
#     check[col_name] = check[col_name].replace({len(combo) / len(combo): True, 0.0: False})

for col in check.columns:
    check[col] = np.round(check[col])
    check[col] = check[col].replace({1: True, 0: False})

In [49]:
model_tr_predictions = [tr1, tr2, tr3, tr4, tr5, tr6]

check_tr = pd.DataFrame({f'model{i+1}': model for i, model in enumerate(model_tr_predictions)})

for col in check_tr.columns:
    check_tr[col] = check_tr[col].replace({True: 1, False: 0})
    
check_tr['model6'] = np.round(check_tr['model6'])

for combo in combinations:
    col_name = f'composition_{"".join(map(str, combo))}'
    check_tr[col_name] = check_tr[list(map(lambda x: f'model{x}', combo))].mean(axis=1)
#     check[col_name] = check[col_name].replace({len(combo) / len(combo): True, 0.0: False})

for col in check_tr.columns:
    check_tr[col] = np.round(check_tr[col])
    check_tr[col] = check_tr[col].replace({1: True, 0: False})

In [50]:
def calculate_accuracy(true_values, predicted_values):
    if len(true_values) != len(predicted_values):
        raise ValueError("Lengths are different")
    correct_predictions = 0
    for true, predicted in zip(true_values, predicted_values):
        if true == predicted:
            correct_predictions += 1
    accuracy = correct_predictions / len(true_values)
    return accuracy

In [51]:
# Define the list of model names
model_names = list(check_tr.columns[:6])

# Iterate through the model names
for model_name in model_names:
    train_accuracy = calculate_accuracy(y_train_copy, check_tr[model_name].replace({3/3:True, 2/3:True, 1/3:False, 0/3:False}))
    valid_accuracy = calculate_accuracy(final_valid_test, check[model_name].replace({3/3:True, 2/3:True, 1/3:False, 0/3:False}))
    
    print(f"Model {model_name[-1]}. Accuracy (train/valid): {train_accuracy:.5f} / {valid_accuracy:.5f}")

# Define the list of composition names
composition_names = list(check_tr.columns[6:])

# Iterate through the composition names
for composition_name in composition_names:
    train_accuracy = calculate_accuracy(y_train_copy, check_tr[composition_name])
    valid_accuracy = calculate_accuracy(final_valid_test, check[composition_name])
    
    print(f"{composition_name}. Accuracy (train/valid): {train_accuracy:.5f} / {valid_accuracy:.5f}")


Model 1. Accuracy (train/valid): 0.92724 / 0.76724
Model 2. Accuracy (train/valid): 0.85864 / 0.79598
Model 3. Accuracy (train/valid): 0.86080 / 0.65805
Model 4. Accuracy (train/valid): 0.78243 / 0.73851
Model 5. Accuracy (train/valid): 0.75827 / 0.74425
Model 6. Accuracy (train/valid): 0.81406 / 0.80172
composition_123. Accuracy (train/valid): 0.89574 / 0.79310
composition_124. Accuracy (train/valid): 0.86569 / 0.79023
composition_125. Accuracy (train/valid): 0.86713 / 0.78448
composition_126. Accuracy (train/valid): 0.85835 / 0.79885
composition_134. Accuracy (train/valid): 0.87460 / 0.75287
composition_135. Accuracy (train/valid): 0.88021 / 0.75575
composition_136. Accuracy (train/valid): 0.88050 / 0.78736
composition_145. Accuracy (train/valid): 0.84570 / 0.75287
composition_146. Accuracy (train/valid): 0.83434 / 0.79598
composition_156. Accuracy (train/valid): 0.84052 / 0.79310
composition_234. Accuracy (train/valid): 0.83995 / 0.75000
composition_235. Accuracy (train/valid): 0.84

# Creating final submission

In [52]:
temp = output_1.copy()
temp['Transported_2'] = output_2.Transported
temp['Transported_3'] = output_3.Transported
temp['Transported_4'] = output_4.Transported
temp['Transported_5'] = output_5.Transported
temp['Transported_6'] = output_lgb.Transported


# temp['sum'] = (temp.Transported + temp.Transported_2 + temp.Transported_3 + temp.Transported_4 + temp.Transported_5)/5
# temp['sum'] = temp['sum'].replace({5/5: True, 4/5: True, 3/5:True, 2/5:False, 1/5:False, 0/5:False})

temp['sum'] = (temp.Transported + temp.Transported_3 + temp.Transported_6)/3
temp['sum'] = temp['sum'].replace({3/3:True, 2/3:True, 1/3:False, 0/3:False})

output = output_1.copy()
output.Transported = temp['sum']
output.to_csv('submission_136.csv',index=False)



temp['sum'] = (temp.Transported_2 + temp.Transported_3 + temp.Transported_6)/3
temp['sum'] = temp['sum'].replace({3/3:True, 2/3:True, 1/3:False, 0/3:False})

output = output_1.copy()
output.Transported = temp['sum']
output.to_csv('submission_236.csv',index=False)

In [53]:
# output = output_1.copy()
# output['Transported'] = temp['Transported_6'].replace({1: True, 0: False})
# output.to_csv('submission.csv',index=False)
# output